In [0]:
#Importacion de librerias y funciones

from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text('catalogo_source','catalog_dev')
dbutils.widgets.text('schema_source','silver')
dbutils.widgets.text('schema_sink','golden')

In [0]:
catalogo = dbutils.widgets.get('catalogo_source')
schema_source = dbutils.widgets.get('schema_source')
schema_sink = dbutils.widgets.get('schema_sink')

In [0]:
# lectura de la data
df_Departamentos = spark.table(f"{catalogo}.{schema_source}.TblDepartamentos")
df_PuestoTrabajo = spark.table(f"{catalogo}.{schema_source}.TblPuestosTrabajo")
df_Empleados = spark.table(f"{catalogo}.{schema_source}.TblEmpleados")


In [0]:
# 1 Unir empleados con puestos

emp_puesto = df_Empleados.join(df_PuestoTrabajo, on="IdPuesto").selectExpr([f"{col} as {col}_emp" for col in df_Empleados.columns] + [f"{col} as {col}_puesto" for col in df_PuestoTrabajo.columns])

In [0]:
# 2. Unir el resultado con departamentos
resultado = emp_puesto.join(df_Departamentos, emp_puesto["IdDepartamento_puesto"] == df_Departamentos["IdDepartamento"], how='inner')

In [0]:
resultado.display()

In [0]:
df_Final = resultado

In [0]:
df_Final.display()

In [0]:
# Escribir a nuestra capa gold
df_Final.write.mode('overwrite').format('delta').saveAsTable(f'{catalogo}.{schema_sink}.Empresa')

print()